In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}','volbuy':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}','sell_price':'{:.2f}','diff':'{:.2f}','cost':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}',
    'amount':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
    'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','pct':'{:,.2f}%'
}
cols = 'trade name qty price reason market active'.split()

today = date.today()
print(today)

2025-07-28


In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
#yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-07-28
yesterday: 2025-07-25 00:00:00


In [3]:
# find the beginning of the week for the given yesterday
week_start = yesterday.to_period('W').start_time
week_end = yesterday.to_period('W').end_time
week_start = week_start.date()
week_end = week_end.date()
print(f'week start: {week_start}')
print(f'week end: {week_end}')

week start: 2025-07-21
week end: 2025-07-27


### Restart and Run All Cells

In [5]:
sql = '''
SELECT name, buys.date AS buy_date, sells.date AS sell_date,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE sells.date BETWEEN '%s' AND '%s'
ORDER BY name, sells.date'''
sql = sql % (week_start, week_end)
print(sql)


SELECT name, buys.date AS buy_date, sells.date AS sell_date,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE sells.date BETWEEN '2025-07-21' AND '2025-07-27'
ORDER BY name, sells.date


In [6]:
sells_df = pd.read_sql(sql, conpf)
sells_df.style.format(format_dict)

,name,buy_date,sell_date,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,profit
0,KCE,2021-10-07,2025-07-24,22.40,87.00,-64.60,"1,000","22,400.00","87,000.00","-64,600.00",-74.25%,"-64,842.32"


In [7]:
sells_grp = sells_df.groupby(['name'])[['qty','sell_amt','buy_amt','gross','profit']].sum()
#sells_grp.style.format(format_dict)
sells_grp

,qty,sell_amt,buy_amt,gross,profit
name,,,,,
KCE,1000,22400.0,87000.0,-64600.0,-64842.32


In [8]:
sells_df[['sell_amt','buy_amt','gross','profit']].sum()

sell_amt    22400.00
buy_amt     87000.00
gross      -64600.00
profit     -64842.32
dtype: float64

In [9]:
sql = '''
SELECT name, buys.date AS buy_date, buys.price AS buy_price, 
qty, (buys.price * qty) AS buy_amt 
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE buys.date BETWEEN '%s' AND '%s'
ORDER BY name, buys.date'''
sql = sql % (week_start, week_end)
print(sql)


SELECT name, buys.date AS buy_date, buys.price AS buy_price, 
qty, (buys.price * qty) AS buy_amt 
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE buys.date BETWEEN '2025-07-21' AND '2025-07-27'
ORDER BY name, buys.date


In [10]:
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,buy_date,buy_price,qty,buy_amt


In [11]:
buys_df[['buy_amt']].sum()

buy_amt    0
dtype: object

In [12]:
sells_df[['buy_amt']].sum()

buy_amt    87000.0
dtype: float64

In [13]:
diff_amt = buys_df[['buy_amt']].sum() - sells_df[['buy_amt']].sum()
diff_amt

buy_amt   -87000.0
dtype: object

In [14]:
sql = '''
SELECT name, available_qty, cost, status
FROM stocks
'''
stocks_df = pd.read_sql(sql, conlite)
stocks_df.set_index('name', inplace=True)
stocks_df.style.format(format_dict)

,available_qty,cost,status
name,,,
AH,"1,200",37.00,X
AIMIRT,"12,500",10.90,S
AWC,"9,000",4.96,X
BCH,"4,000",21.70,X
CPNREIT,"55,000",18.00,X
DIF,"40,000",13.10,B
GVREIT,"75,000",7.64,B
IVL,"9,000",36.00,S
3BBIF,"120,000",10.10,X


In [15]:
df_merge = pd.merge(sells_grp,stocks_df,on='name',how='inner')
df_merge.style.format(format_dict)

,qty,sell_amt,buy_amt,gross,profit,available_qty,cost,status
name,,,,,,,,
KCE,"1,000","22,400.00","87,000.00","-64,600.00","-64,842.32","1,000",87.00,S


In [16]:
sql = '''
SELECT name, SUM(volbuy) AS volbuy, price
FROM buy
WHERE active = 1
GROUP BY name
ORDER BY name'''
buy_df = pd.read_sql(sql, const)
buy_df.volbuy = buy_df.volbuy.astype(int)
buy_df.style.format(format_dict)

,name,volbuy,price
0,3BBIF,"120,000",10.10
1,AH,"1,200",37.00
2,AIMIRT,"12,500",10.90
3,AWC,"9,000",4.96
4,BCH,"4,000",21.70
5,CPNREIT,"55,000",18.00
6,DIF,"40,000",13.10
7,GVREIT,"75,000",7.64
8,IVL,"9,000",36.00
9,JMART,"6,800",33.00


In [17]:
df_merge2 = pd.merge(buy_df,stocks_df,on='name',how='inner')
df_merge2.style.format(format_dict)

,name,volbuy,price,available_qty,cost,status
0,3BBIF,"120,000",10.10,"120,000",10.10,X
1,AH,"1,200",37.00,"1,200",37.00,X
2,AIMIRT,"12,500",10.90,"12,500",10.90,S
3,AWC,"9,000",4.96,"9,000",4.96,X
4,BCH,"4,000",21.70,"4,000",21.70,X
5,CPNREIT,"55,000",18.00,"55,000",18.00,X
6,DIF,"40,000",13.10,"40,000",13.10,B
7,GVREIT,"75,000",7.64,"75,000",7.64,B
8,IVL,"9,000",36.00,"9,000",36.00,S
9,JMART,"6,800",33.00,"6,800",33.00,X


In [18]:
mask1 = df_merge2.volbuy != df_merge2.available_qty
mask2 = df_merge2.price != df_merge2.cost
df_merge2[mask1 | mask2].style.format(format_dict)

,name,volbuy,price,available_qty,cost,status


### New process

In [20]:
sql = '''
SELECT trade, O.name, O.qty, price, S.qty, buy_target
FROM orders O
JOIN stocks S
ON O.name = S.name
WHERE trade = "B" AND price <> buy_target
ORDER BY trade, O.name
'''
orders_df = pd.read_sql(sql, conlite)
orders_df.style.format(format_dict)

,trade,name,qty,price,qty,buy_target


In [21]:
sql = '''
SELECT trade, O.name, O.qty, price, S.qty, sell_target
FROM orders O
JOIN stocks S
ON O.name = S.name
WHERE trade = "S" AND price <> sell_target
'''
orders_df = pd.read_sql(sql, conlite)
orders_df.style.format(format_dict)

,trade,name,qty,price,qty,sell_target
0,S,WHAIR,"10,000",5.50,"10,000",8.05
1,S,KCE,"1,000",22.90,"1,000",22.20


In [22]:
const.close()
conmy.close()
conlite.close()
conpf.close()

In [23]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-07-28 20:19:09
